In [4]:
import pandas as pd
import folium
from folium.plugins import HeatMap

In [2]:

df = pd.read_csv('/Users/pranav_medikonduru/Downloads/USGS_query.csv')

print(df.head())

                       time   latitude   longitude  depth   mag magType  \
0  2025-01-26T14:03:01.190Z  40.334833 -124.942833   7.04  4.13      mw   
1  2025-01-02T02:34:04.660Z  38.845500 -122.756333   1.42  4.65      mw   
2  2024-12-17T01:08:14.980Z  40.282000 -123.710667  28.97  4.05      mw   
3  2024-12-10T13:01:07.270Z  40.332500 -124.792000   5.82  4.17      mw   
4  2024-12-10T00:39:56.966Z  39.168600 -119.022500   8.10  4.20      ml   

     nst     gap      dmin     rms  ...                   updated  \
0  177.0  231.00  0.461300  0.3000  ...  2025-01-30T22:40:12.722Z   
1  115.0   44.00  0.009316  0.0800  ...  2025-02-02T16:55:45.589Z   
2   79.0   23.00  0.079750  0.1800  ...  2025-02-04T01:18:21.392Z   
3  158.0  226.00  0.346500  0.3200  ...  2025-01-17T16:58:40.613Z   
4   28.0   42.03  0.250000  0.1405  ...  2025-01-07T03:40:10.291Z   

                            place        type horizontalError depthError  \
0       56 km WNW of Petrolia, CA  earthquake            0

In [3]:
# ----------------------------
# Map 1: Earthquake Locations
# ----------------------------

# Define a center point for the US.
us_center = [39.8283, -98.5795]

# Create a folium map with a clean base.
eq_map = folium.Map(location=us_center, zoom_start=4, tiles='CartoDB positron')

# Loop through the data and add a marker for each earthquake.
for idx, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    
    mag = row.get('mag', 0)
    popup_text = f"Magnitude: {mag}" if pd.notnull(mag) else "Earthquake event"
    
    # Create a circle marker. The marker’s radius increases with magnitude.
    folium.CircleMarker(
        location=[lat, lon],
        radius=3 + (mag if pd.notnull(mag) else 0) * 2,
        popup=popup_text,
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(eq_map)

# Save the earthquake locations map to an HTML file.
eq_map.save("earthquake_locations.html")


In [ ]:
# ---------------------------------------------------------
# Map 2: Heatmap for Areas Likely Affected by Future Quakes
# ---------------------------------------------------------

heat_data = [
    [row['latitude'], row['longitude'], row['mag'] if pd.notnull(row['mag']) else 1]
    for idx, row in df.iterrows()
]

# Create a base map.
heat_map = folium.Map(location=us_center, zoom_start=4, tiles='CartoDB positron')

# Add the heatmap overlay.
HeatMap(heat_data, min_opacity=0.5, radius=15, blur=25, max_zoom=6).add_to(heat_map)

# Save the heatmap to an HTML file.
heat_map.save("earthquake_heatmap.html")


In [ ]:

# ---------------------------------------------------------
# Map 3: Clustering for Future Risk Prediction
# ---------------------------------------------------------

from sklearn.cluster import KMeans
import numpy as np

# Filter out any rows with missing location data.
coords = df[['latitude', 'longitude']].dropna()

num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(df[['latitude', 'longitude']])

# Aggregate data by cluster: calculate the mean latitude, longitude, and magnitude.
cluster_summary = df.groupby('cluster').agg({
    'latitude': 'mean',
    'longitude': 'mean',
    'mag': 'mean'
}).reset_index()

# Create a new map to display the clusters.
cluster_map = folium.Map(location=us_center, zoom_start=4, tiles='CartoDB positron')
for idx, row in cluster_summary.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10 + row['mag'] * 2,  
        popup=f"Predicted Magnitude: {row['mag']:.2f}",
        color='red',
        fill=True,
        fill_opacity=0.6
    ).add_to(cluster_map)

# Save the cluster map.
cluster_map.save("earthquake_cluster_map.html")
